In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dis
import time
import threading

from circular_queue.unsafe import UnsafeCircularQueue
from circular_queue.deadlock import DeadLockQueue
from circular_queue.safe import ThreadSafeCircularQueue

from dinning_philosopher.fork import Fork
from dinning_philosopher.philosopher import DeadLockPhilosopher
from dinning_philosopher.philosopher import ServedPhilosopher

from singleton import singleton
from singleton import optimized_singleton

# Race Conditions

In [3]:
def unsafe_increment(total):
    global n
    for i in range(total):
        n += 1

In [4]:
n = 0
unsafe_increment(1000000)
unsafe_increment(1000000)
print(n)

2000000


In [5]:
n = 0
thread1 = threading.Thread(target=unsafe_increment, args=(1000000,))
thread2 = threading.Thread(target=unsafe_increment, args=(1000000,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(n)

1478759


`n += 1` 字节码：

In [6]:
import dis
def foo():
    global n
    n += 1
dis.dis(foo, depth=0)

  4           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


# 锁

In [7]:
def safe_increment(total, lock):
    global n
    for i in range(total):
        with lock:
            n += 1


n = 0
lock = threading.Lock()
thread1 = threading.Thread(target=safe_increment, args=(1000000, lock))
thread2 = threading.Thread(target=safe_increment, args=(1000000, lock))

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(n)

2000000


## 死锁

### 生产者/消费者

In [8]:
def consume(queue, total):
    for i in range(total):
        print(i+1, queue.remove())
        time.sleep(0.1)

def produce(queue, total, message):
    for i in range(total):
        queue.add('{:d}: {:s}'.format(i, message))
        time.sleep(0.1)

### 线程不安全的队列

In [9]:
unsafe_queue = UnsafeCircularQueue(10)

thread1 = threading.Thread(target=produce, args=(unsafe_queue, 100, "Hello!"))
thread2 = threading.Thread(target=produce, args=(unsafe_queue, 100, "Goodbye!"))
thread3 = threading.Thread(target=consume, args=(unsafe_queue, 200))

thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()

1 0: Hello!
2 0: Goodbye!
3 1: Hello!
4 1: Goodbye!
5 2: Hello!
6 2: Goodbye!
7 3: Hello!
8 3: Goodbye!
9 4: Hello!
10 9: Hello!
11 10: Goodbye!
12 10: Hello!
13 11: Goodbye!
14 11: Hello!
15 12: Goodbye!
16 12: Hello!
17 13: Goodbye!
18 13: Hello!
19 14: Goodbye!
20 19: Hello!
21 20: Goodbye!
22 20: Hello!
23 21: Goodbye!
24 21: Hello!
25 22: Goodbye!
26 22: Hello!
27 23: Goodbye!
28 23: Hello!
29 24: Goodbye!
30 29: Hello!
31 30: Hello!
32 30: Goodbye!
33 31: Goodbye!
34 31: Hello!
35 32: Goodbye!
36 32: Hello!
37 33: Goodbye!
38 33: Hello!
39 34: Goodbye!
40 39: Hello!
41 40: Goodbye!
42 40: Hello!
43 41: Goodbye!
44 41: Hello!
45 42: Goodbye!
46 42: Hello!
47 43: Goodbye!
48 43: Hello!
49 44: Goodbye!
50 49: Hello!
51 50: Goodbye!
52 50: Hello!
53 51: Hello!
54 51: Goodbye!
55 52: Hello!
56 52: Goodbye!
57 53: Hello!
58 53: Goodbye!
59 54: Hello!
60 59: Hello!
61 60: Goodbye!
62 60: Hello!
63 61: Goodbye!
64 61: Hello!
65 62: Goodbye!
66 62: Hello!
67 63: Goodbye!
68 63: Hello!
69 

### 死锁队列

运行下段代码会导致程序阻塞住，直接中断即可。

In [10]:
dead_queue = DeadLockQueue(10)

thread1 = threading.Thread(target=produce, args=(dead_queue, 100, "Hello!"))
thread2 = threading.Thread(target=produce, args=(dead_queue, 100, "Goodbye!"))
thread3 = threading.Thread(target=consume, args=(dead_queue, 200))

thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()

1 0: Hello!
2 0: Goodbye!
3 1: Hello!
4 1: Goodbye!
5 2: Hello!
6 2: Goodbye!
7 3: Hello!
8 3: Goodbye!
9 4: Hello!


KeyboardInterrupt: 

### 线程安全队列

使用 `Condition` 的一个线程安全的队列实现。

In [11]:
queue = ThreadSafeCircularQueue(10)

thread1 = threading.Thread(target=produce, args=(queue, 100, "Hello!"))
thread2 = threading.Thread(target=produce, args=(queue, 100, "Goodbye!"))
thread3 = threading.Thread(target=consume, args=(queue, 200))

thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()

1 0: Hello!
2 0: Goodbye!
3 1: Hello!
4 1: Goodbye!
5 2: Hello!
6 2: Goodbye!
7 3: Hello!
8 3: Goodbye!
9 4: Hello!
10 4: Goodbye!
11 5: Hello!
12 5: Goodbye!
13 6: Hello!
14 6: Goodbye!
15 7: Hello!
16 7: Goodbye!
17 8: Hello!
18 8: Goodbye!
19 9: Hello!
20 9: Goodbye!
21 10: Hello!
22 11: Hello!
23 10: Goodbye!
24 11: Goodbye!
25 12: Goodbye!
26 12: Hello!
27 13: Goodbye!
28 13: Hello!
29 14: Goodbye!
30 14: Hello!
31 15: Goodbye!
32 16: Goodbye!
33 15: Hello!
34 17: Goodbye!
35 16: Hello!
36 17: Hello!
37 18: Goodbye!
38 18: Hello!
39 19: Goodbye!
40 20: Goodbye!
41 19: Hello!
42 21: Goodbye!
43 22: Goodbye!
44 23: Goodbye!
45 20: Hello!
46 24: Goodbye!
47 25: Goodbye!
48 26: Goodbye!
49 21: Hello!
50 22: Hello!
51 27: Goodbye!
52 23: Hello!
53 28: Goodbye!
54 24: Hello!
55 25: Hello!
56 29: Goodbye!
57 26: Hello!
58 27: Hello!
59 30: Goodbye!
60 31: Goodbye!
61 32: Goodbye!
62 33: Goodbye!
63 28: Hello!
64 29: Hello!
65 34: Goodbye!
66 30: Hello!
67 31: Hello!
68 35: Goodbye!
69 36

# 应用：哲学家就餐问题

In [12]:
def generate_forks(n):
    forks = list()
    for i in range(n):
        forks.append(Fork(i))
    return forks


def arrange_philosophers(philosophers, forks):
    assert len(philosophers) == len(forks)
    
    n = len(philosophers)
    for i in range(n):
        left_fork = forks[(i + 1) % n]
        right_fork = forks[i]
        philosophers[i].set_left_fork(left_fork)
        philosophers[i].set_right_fork(right_fork)
        print(philosophers[i])


def dinning_philosophers_main(philosophers):
    try:
        threads = list()
        for phil in philosophers:
            t = threading.Thread(target=phil)
            threads.append(t)
            t.start()
        for t in threads:
            t.join()
    except KeyboardInterrupt:
        for t in threads:
            t.is_alive = False

NUM_PHILOSOPHERS = 5

## 死锁解法

In [13]:
forks = generate_forks(NUM_PHILOSOPHERS)
philosophers = list()
for i in range(NUM_PHILOSOPHERS):
    philosophers.append(DeadLockPhilosopher(i))
arrange_philosophers(philosophers, forks)

DeadLockPhilosopher[index=0, left_fork=Fork[item_index=1], right_fork=Fork[item_index=0]]
DeadLockPhilosopher[index=1, left_fork=Fork[item_index=2], right_fork=Fork[item_index=1]]
DeadLockPhilosopher[index=2, left_fork=Fork[item_index=3], right_fork=Fork[item_index=2]]
DeadLockPhilosopher[index=3, left_fork=Fork[item_index=4], right_fork=Fork[item_index=3]]
DeadLockPhilosopher[index=4, left_fork=Fork[item_index=0], right_fork=Fork[item_index=4]]


下一行代码会由于死锁完全阻塞住，直接终止下一行代码的运行即可。

In [14]:
dinning_philosophers_main(philosophers)

Philosopher #0 is taking fork #1.
Philosopher #1 is taking fork #2.
Philosopher #2 is taking fork #3.
Philosopher #3 is taking fork #4.
Philosopher #4 is taking fork #0.


## 服务生法

In [15]:
waiter = threading.Lock()
forks = generate_forks(NUM_PHILOSOPHERS)
philosophers = list()
for i in range(NUM_PHILOSOPHERS):
    philosophers.append(ServedPhilosopher(i, waiter))
arrange_philosophers(philosophers, forks)

ServedPhilosopher[index=0, left_fork=Fork[item_index=1], right_fork=Fork[item_index=0]][waiter=<unlocked _thread.lock object at 0x7f868073c510>]
ServedPhilosopher[index=1, left_fork=Fork[item_index=2], right_fork=Fork[item_index=1]][waiter=<unlocked _thread.lock object at 0x7f868073c510>]
ServedPhilosopher[index=2, left_fork=Fork[item_index=3], right_fork=Fork[item_index=2]][waiter=<unlocked _thread.lock object at 0x7f868073c510>]
ServedPhilosopher[index=3, left_fork=Fork[item_index=4], right_fork=Fork[item_index=3]][waiter=<unlocked _thread.lock object at 0x7f868073c510>]
ServedPhilosopher[index=4, left_fork=Fork[item_index=0], right_fork=Fork[item_index=4]][waiter=<unlocked _thread.lock object at 0x7f868073c510>]


In [16]:
dinning_philosophers_main(philosophers)

Philosopher #0 is taking fork #1.
Philosopher #0 is taking fork #0.
Philosopher #0 is eating.(1 totally)Philosopher #1 is taking fork #2.

Philosopher #0 is putting down fork #1.
Philosopher #0 is putting down fork #0.
Philosopher #0 is thinking.(1 totally)Philosopher #1 is taking fork #1.

Philosopher #1 is eating.(1 totally)Philosopher #2 is taking fork #3.

Philosopher #1 is putting down fork #2.
Philosopher #1 is putting down fork #1.
Philosopher #1 is thinking.(1 totally)
Philosopher #2 is taking fork #2.
Philosopher #2 is eating.(1 totally)
Philosopher #3 is taking fork #4.
Philosopher #2 is putting down fork #3.
Philosopher #2 is putting down fork #2.
Philosopher #2 is thinking.(1 totally)
Philosopher #3 is taking fork #3.
Philosopher #3 is eating.(1 totally)Philosopher #4 is taking fork #0.

Philosopher #3 is putting down fork #4.
Philosopher #3 is putting down fork #3.
Philosopher #3 is thinking.(1 totally)
Philosopher #4 is taking fork #4.
Philosopher #4 is eating.(1 totally)

# 应用：线程安全 singleton

In [17]:
@singleton
class DummyClient:
    
    def __init__(self):
        print('DummyClient is created..')


@optimized_singleton
class DummyClinetV2:
    
    def __init__(self):
        print('DummyClientV2 is created..')

In [18]:
%timeit DummyClient()

DummyClient is created..
282 ns ± 1.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
%timeit DummyClinetV2()

DummyClientV2 is created..
130 ns ± 0.408 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
